In [1]:
import pandas as pd
import numpy as np
import unicodedata 

In [2]:
def remove_special_and_accented_characters(df):
    cleaned_df = df.copy()
    for col in cleaned_df.columns:
        if cleaned_df[col].dtype == 'O':  # 'O' represents object data type (strings)
            cleaned_df[col] = cleaned_df[col].apply(lambda x: ''.join(char for char in unicodedata.normalize('NFKD', str(x)) if unicodedata.category(char) != 'Mn'))
    return cleaned_df

# Dependent variable - importing NGO density information - health facilities

Number of NGO at SUS unified health facilities management system: DATASUS http://tabnet.datasus.gov.br/cgi/deftohtm.exe?cnes/cnv/estabbr.def

In [6]:
CNESOSC = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\SUS\CNESosc.csv', engine='python', delimiter=";", index_col=False, encoding='latin1')
CNESOSC['IBGE'] = CNESOSC['IBGE'].astype(int)

In [7]:
CNESOSC.fillna(0, inplace=True)

In [8]:
CNESOSC

,IBGE,CNESESTOSC2014,CNESESTOSC2015,CNESESTOSC2016,CNESESTOSC2017,CNESESTOSC2018,CNESESTOSC2019,CNESESTOSC2020
0,110001,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,110002,2.0,2.0,3.0,3.0,4.0,5.0,5.0
2,110003,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110004,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,110005,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
2527,522060,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2528,522140,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2529,522160,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2530,522200,0.0,0.0,0.0,0.0,1.0,1.0,1.0


# Control variables -  pib and other info: population and pib percapita

Municipality Identification info: source: IBGE https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html?=&t=acesso-ao-produto

Population: source IBGE https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?=&t=resultados

Economic indicators: source: IBGE https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html?=&t=downloads


In [111]:
dadosibge = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\IBGE\dadosibge.csv', encoding = 'latin1', delimiter=";", index_col=False, dtype={'IBGE': np.int64,'Municipio': object})
dadosibge['IBGE'] = dadosibge['IBGE'].astype(int)
dadosibge['IBGE7'] = dadosibge['IBGE7'].astype(int)
dadosibge['MUNICIPIO'] = dadosibge['MUNICIPIO'].astype(str)

In [112]:
dadosibge['MunHierarchy'] = np.select(
    [
        dadosibge['Centrolocal'] == 1,
        dadosibge['CentroSubregional'] == 1,
        dadosibge['CapitalRegional'] == 1,
        dadosibge['CentroZona'] == 1,
              dadosibge['Nucleo'] == 1
    ],
    [
        'Local center',
        'Subregional center',
        'Regional Capital',
        'Zone center',
        'Core'
    ],
    default='No Category'  # You can set a default value if none of the conditions are met
)


In [113]:
Pop = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\IBGE\Populacao2015a2020.csv', engine = 'python',  delimiter=";", index_col=False, dtype={'IBGE7': np.int64})
Pop['IBGE7'] = Pop['IBGE7'].astype(int)


In [114]:
Eco = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\IBGE\Eco2015to2020.csv',    delimiter=";")
Eco['IBGE7'] = Eco['IBGE7'].astype(int)

In [115]:
columns_to_keep = ['IBGE7', 'Pib2014','Pib2015','Pib2016','Pib2017','Pib2018','Pib2019','Pib2020', 
                   'Pibpercapita2015','Pibpercapita2016','Pibpercapita2017', 
                   'Pibpercapita2018', 'Pibpercapita2019', 'Pibpercapita2020']


Eco = Eco[columns_to_keep]

In [116]:
Eco

,IBGE7,Pib2014,Pib2015,Pib2016,Pib2017,Pib2018,Pib2019,Pib2020,Pibpercapita2015,Pibpercapita2016,Pibpercapita2017,Pibpercapita2018,Pibpercapita2019,Pibpercapita2020
0,1100015,4.213000e+05,421300,478217,485374,498980,495629,570272,16471,18749,19081,21538,21601,25091
1,1100023,2.037799e+06,2037799,2184346,2287910,2464704,2578830,2818049,19519,20627,21314,23215,23908,25730
2,1100031,1.165652e+05,116565,133342,138110,140503,139959,167190,18342,21202,22190,25837,26348,32226
3,1100049,1.794478e+06,1794478,1947283,2082761,2175840,2260644,2519353,20573,22159,23532,25655,26484,29331
4,1100056,3.977361e+05,397736,408194,439245,470647,506854,600670,22114,22729,24492,28621,31052,37069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,3.123479e+05,312348,373802,395861,449973,483663,587313,23213,27552,28948,32735,34889,42020
5566,5222054,3.222984e+05,322298,351750,350839,339089,342356,418557,39444,42451,41781,39379,39158,47172
5567,5222203,1.112289e+05,111229,138810,118639,108469,113668,121887,20242,24721,20701,18000,18420,19310
5568,5222302,1.989352e+05,198935,210475,182203,179915,206971,218473,35664,37351,32022,31246,35556,37143


# Control variable - poverty

  

Poverty: source CADUNICO https://api.pgi.gov.br/api/1/serie/209482.json

        

In [15]:
Cadunico =  pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\Cadúnico\CadUnico.csv', engine = 'python',  delimiter=";", index_col=False, dtype={'IBGE': np.int64})
Cadunico['IBGE'] = Cadunico['IBGE'].astype(int)

In [16]:
Cadunico

,IBGE,CadUnico2015,CadUnico2016,CadUnico2017,CadUnico2018,CadUnico2019,CadUnico2020
0,110001,8172,6617,6617,5484,5433,8172
1,110002,22165,17500,17500,15105,14339,22165
2,110003,1266,879,879,684,712,1266
3,110004,15419,11492,11492,10733,11283,15419
4,110005,4015,3024,3024,2650,2650,4015
...,...,...,...,...,...,...,...
5565,522200,2589,1609,1609,913,1261,2589
5566,522205,2492,2546,2546,2579,2583,2492
5567,522220,2578,2331,2331,1999,1900,2578
5568,522230,2382,2232,2232,2093,1914,2382


# Independent variable - paid health amendments to the federal budget

National Health Fund transfer for individual ammendments - only OSC and only public: source: FNS https://infoms.saude.gov.br/extensions/CGIN_Painel_Emendas/CGIN_Painel_Emendas.html#GUIA02 

In [17]:
FNStransfersOSC= pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\SUS\FNStransfersOSC.csv',  engine = 'python',  delimiter=";", index_col=False)
FNStransfersOSC['IBGE'] = FNStransfersOSC['IBGE'].astype(int)

In [18]:
pivot_FNStransfersOSC = FNStransfersOSC.pivot_table(
    index='IBGE',
    columns=['Ano'],
    values=['EMPENHADOOSCFNS', 'PAGOOSCFNS'],
    aggfunc='sum',
    fill_value=0
)


In [19]:
pivot_FNStransfersOSC.columns = pivot_FNStransfersOSC.columns.map(lambda x: ''.join(map(str, x)))

In [20]:
pivot_FNStransfersOSC = pivot_FNStransfersOSC.reset_index()

In [21]:
pivot_FNStransfersOSC = pivot_FNStransfersOSC.apply(pd.to_numeric, errors='coerce')

In [22]:
pivot_FNStransfersOSC.fillna(0, inplace=True)

In [23]:
pivot_FNStransfersOSC

,IBGE,EMPENHADOOSCFNS2014,EMPENHADOOSCFNS2015,EMPENHADOOSCFNS2016,EMPENHADOOSCFNS2017,EMPENHADOOSCFNS2018,EMPENHADOOSCFNS2019,EMPENHADOOSCFNS2020,EMPENHADOOSCFNS2021,EMPENHADOOSCFNS2022,...,PAGOOSCFNS2014,PAGOOSCFNS2015,PAGOOSCFNS2016,PAGOOSCFNS2017,PAGOOSCFNS2018,PAGOOSCFNS2019,PAGOOSCFNS2020,PAGOOSCFNS2021,PAGOOSCFNS2022,PAGOOSCFNS2023
0,110004,0,0,0,0,0,0,0,300000,0,...,0,0,0,0,0,0,0,300000,0,0
1,110009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,96252
2,110010,0,0,0,0,0,400000,0,0,81000,...,0,0,0,0,0,400000,0,0,81000,0
3,110012,600000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,110020,2942992,1997112,7564256,0,0,0,0,400000,0,...,942992,0,2843038,0,0,0,0,340000,0,13064072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,521880,570000,424513,1634983,1024960,1499850,2549950,2153035,2459000,3039329,...,250000,174543,484983,979610,1333914,2132050,1887279,2398474,2999606,3969480
1324,522020,0,0,99910,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1325,522045,0,0,0,0,0,0,0,100000,0,...,0,0,0,0,0,0,0,45766,0,0
1326,522140,1266998,849887,1352706,499875,699960,949860,439956,649905,150000,...,199998,729889,250000,186968,372149,814140,342053,430503,99793,0


# Control variable - resources from other sources - government funding to non profits

In [24]:
Siga2015 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2015.csv',  engine = 'python',  delimiter=";", index_col=False,  dtype={'Emenda': object,'Empenhado': object, 'Pago': object,'Favorecido (Cod)':object})
Siga2015['Ano'] = 2015

In [25]:
Siga2016 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2016.csv',  engine = 'python',  delimiter=";", index_col=False,  dtype={'Emenda': object,'Empenhado': object, 'Pago': object,'Favorecido (Cod)':object})
Siga2016['Ano'] = 2016

In [26]:
Siga2017 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2017.csv',  engine = 'python',  delimiter=";", index_col=False,  dtype={'Emenda': object,'Empenhado': object, 'Pago': object,'Favorecido (Cod)':object})
Siga2017['Ano'] = 2017

In [27]:
Siga2018 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2018.csv',  engine = 'python',  delimiter=";", index_col=False,   dtype={'Emenda': object,'Empenhado': object, 'Pago': object,'Favorecido (Cod)':object})
Siga2018['Ano'] = 2018

In [28]:
Siga2019 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2019.csv',  engine = 'python',  delimiter=";", index_col=False,  dtype={'Emenda': object,'Empenhado': object, 'Pago': object,'Favorecido (Cod)':object})
Siga2019['Ano'] = 2019

In [29]:
Siga2020 = pd.read_csv('G:\Meu Drive\Doutorado\Disciplinas\Paper_december\Siga2020.csv', engine='python', delimiter=";", index_col=False, dtype={'Emenda': object, 'Empenhado': object, 'Pago': object, 'Favorecido (Cod)': object})
Siga2020['Ano'] = 2020

In [30]:
SIGABASE1 = pd.concat([Siga2015,Siga2016,Siga2017,Siga2018,Siga2019,Siga2020], ignore_index=True, axis=0)

In [31]:
SIGABASE1

,Localidade.UF,Localidade.Localidade,Localidade.Localidade.Localidade (Cod),Localidade (Tipo),Órgão (Cod/Desc),Funcional,Emenda,Resultado EOF (Cod/Desc),Favorecido (Cod),Favorecido,Favorecido (Tipo),Localidade (Favorecido).UF,Localidade (Favorecido).Localidade,Localidade (Favorecido).Localidade.Localidade (Cod),Empenhado,Pago,Ano
0,AC,ACRE (ESTADO),1200000,ACRE (ESTADO),14000 - JUSTIÇA ELEITORAL,02.122.0570.09HB.0012,-1,0 - DESP. FINANCEIRA,170502,SECRETARIA DO TESOURO NACIONAL/COFIN/STN,UNIDADE GESTORA,DF,BRASILIA,5300108,"3.712.731,14","3.711.341,12",2015
1,AC,ACRE (ESTADO),1200000,ACRE (ESTADO),14000 - JUSTIÇA ELEITORAL,02.122.0570.09HB.0012,-1,0 - DESP. FINANCEIRA,NÃO APLICÁVEL,NÃO APLICÁVEL,NÃO APLICÁVEL,NÃO INFORMADO,NÃO INFORMADO,NÃO INFORMADO,"0,00","0,00",2015
2,AC,ACRE (ESTADO),1200000,ACRE (ESTADO),14000 - JUSTIÇA ELEITORAL,02.122.0570.20GP.0012,-1,2 - DESP. PRIM. DISCRICIONÁRIA,00365231000280,DISRI - DISTRIBUIDORA LTDA,PESSOA JURIDICA,AC,FEIJO,1200302,"240,00","0,00",2015
3,AC,ACRE (ESTADO),1200000,ACRE (ESTADO),14000 - JUSTIÇA ELEITORAL,02.122.0570.20GP.0012,-1,2 - DESP. PRIM. DISCRICIONÁRIA,00571529267,UILLIAN DE JESUS FARIAS,PESSOA FISICA,AC,CRUZEIRO DO SUL - AC,1200203,"3.000,00","3.000,00",2015
4,AC,ACRE (ESTADO),1200000,ACRE (ESTADO),14000 - JUSTIÇA ELEITORAL,02.122.0570.20GP.0012,-1,2 - DESP. PRIM. DISCRICIONÁRIA,00578121875,CELSO SANTOS MATHEUS,PESSOA FISICA,AC,RIO BRANCO - AC,1200401,"36.906,48","33.966,41",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792537,TO,XAMBIOA,1722107,MUNICÍPIO,39000 - MINISTÉRIO DA INFRAESTRUTURA,26.782.2087.7L92.0548,-1,3 - DESP. PRIM. DISC. (PAC),11358829000187,AMBIENGER ENGENHARIA AMBIENTAL LTDA,PESSOA JURIDICA,TO,PALMAS - TO,1721000,"0,00","0,00",2020
4792538,TO,XAMBIOA,1722107,MUNICÍPIO,39000 - MINISTÉRIO DA INFRAESTRUTURA,26.782.3006.7L92.0548,-1,2 - DESP. PRIM. DISCRICIONÁRIA,08156424000151,MAIA MELO ENGENHARIA LTDA,PESSOA JURIDICA,PE,RECIFE,2611606,"1.192.263,00","0,00",2020
4792539,TO,XAMBIOA,1722107,MUNICÍPIO,39000 - MINISTÉRIO DA INFRAESTRUTURA,26.782.3006.7L92.0548,-1,2 - DESP. PRIM. DISCRICIONÁRIA,08323347000187,CONSTRUTORA A GASPAR S/A,PESSOA JURIDICA,RN,NATAL,2408102,"20.100.000,00","0,00",2020
4792540,TO,XAMBIOA,1722107,MUNICÍPIO,39000 - MINISTÉRIO DA INFRAESTRUTURA,26.782.3006.7L92.0548,-1,2 - DESP. PRIM. DISCRICIONÁRIA,11358829000187,AMBIENGER ENGENHARIA AMBIENTAL LTDA,PESSOA JURIDICA,TO,PALMAS - TO,1721000,"500.000,00","0,00",2020


In [32]:
def transform_value(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Split the string based on the comma and take the first part
        integer_part = value.split(',')[0]
        return int(integer_part.replace('.', ''))
    else:
        # If the value is already an integer, return it as is
        return value

In [33]:
SIGABASE1['Pago'] = SIGABASE1['Pago'].apply(transform_value)

In [34]:
IPEAoscmap = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\Mapa_OSC\Mapa OSC area_subarea.csv',  engine = 'python',  delimiter=";", index_col=False)


In [35]:
IPEAoscmap['tx_razao_social_osc'] = IPEAoscmap['tx_razao_social_osc'].astype(str)

In [36]:
SIGABASE1['Favorecido'] = SIGABASE1['Favorecido'].astype(str)

In [37]:
Siga = SIGABASE1[SIGABASE1['Favorecido'].isin(IPEAoscmap['tx_razao_social_osc'])]

In [38]:
Siga.rename(columns={'Localidade (Favorecido).Localidade.Localidade (Cod)': 'IBGE7'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [39]:
Sigafiltered = Siga[Siga['Órgão (Cod/Desc)'] == '36000 - MINISTÉRIO DA SAÚDE']

In [40]:
Sigafiltered2 = Sigafiltered[Sigafiltered['Resultado EOF (Cod/Desc)'] != '6 - DESP. PRIM. DISC. (EMENDAS INDIVIDUAIS)']

In [41]:
filtered_indices = Sigafiltered2['Órgão (Cod/Desc)'] == '36000 - MINISTÉRIO DA SAÚDE'
Sigafiltered2.loc[filtered_indices, 'Pago'] = Sigafiltered2.loc[filtered_indices, 'Pago'].apply(transform_value)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [42]:
Sigafiltered2group = Sigafiltered2.groupby(['Ano','IBGE7'])['Pago'].sum().reset_index()
print("Sum after grouping:", Sigafiltered2group['Pago'].sum())

Sum after grouping: 17539190823


In [43]:
Sigafiltered2group = Sigafiltered2group.pivot_table(
    index='IBGE7',
    columns=['Ano'],
    values=['Pago'],
    aggfunc='sum',
    fill_value=0
)


In [44]:
Sigafiltered2group.columns = Sigafiltered2group.columns.map(lambda x: ''.join(map(str, x)))

In [45]:
Sigafiltered2group = Sigafiltered2group.reset_index()

In [46]:
Sigafiltered2group.fillna(0, inplace=True)

In [47]:
Sigafiltered2group['IBGE7'] = Sigafiltered2group['IBGE7'].astype(int)

In [48]:
rename_dict3 = {
    'Pago2015': 'govfunding2015',
    'Pago2016': 'govfunding2016',
    'Pago2017': 'govfunding2017',
    'Pago2018': 'govfunding2018',
    'Pago2019': 'govfunding2019',
    'Pago2020': 'govfunding2020',
    
}

In [49]:
Sigafiltered2group.rename(columns=rename_dict3, inplace=True)

In [50]:
Sigafiltered2group

,IBGE7,govfunding2015,govfunding2016,govfunding2017,govfunding2018,govfunding2019,govfunding2020
0,1100015,0,0,4760,11901,2592,593
1,1100049,3277,3724,5810,5477,36184,34155
2,1100122,0,0,0,0,0,0
3,1100205,4636,0,0,0,0,0
4,1200401,0,0,0,0,0,0
...,...,...,...,...,...,...,...
590,5208707,1745036,58965,600230,656900,1700,17489059
591,5210000,0,0,0,0,0,0
592,5218805,0,0,0,0,0,0
593,5221403,0,0,0,0,0,0


# Moderator - Nonprofit parent organization

Source:https://mapaosc.ipea.gov.br/base-dados

In [148]:
OSCSAUDEPARENT = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\Mapa_OSC\Mapa OSC area_subarea.csv', engine = 'python',  delimiter=";", index_col=False, dtype={'IBGE7': np.int64})
OSCSAUDEPARENT.dropna(subset=['edmu_cd_municipio'], inplace=True)

In [149]:
OSCSAUDEPARENT.rename(columns={'edmu_cd_municipio': 'IBGE7'}, inplace=True)
OSCSAUDEPARENT['IBGE7'] = OSCSAUDEPARENT['IBGE7'].astype(int)

In [150]:
OSCSAUDEPARENT.loc[:, 'cd_identificador_osc'] = OSCSAUDEPARENT['cd_identificador_osc'].astype(str)

In [151]:
OSCSAUDEPARENT['OSCparent'] = OSCSAUDEPARENT['cd_identificador_osc'].str.contains('0001').astype(int)

In [152]:
OSCSAUDEPARENT['ano_fund'] = OSCSAUDEPARENT['dt_fundacao_osc'].astype(str).str[-4:]

In [153]:
OSCSAUDEPARENT['ano_fund'] = pd.to_numeric(OSCSAUDEPARENT['ano_fund'])

In [154]:
OSCSAUDEPARENT = OSCSAUDEPARENT[OSCSAUDEPARENT['saude'] == 1]

In [155]:
OSCSAUDEPARENT['OSCParent2015'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2016, OSCSAUDEPARENT['OSCparent'], np.nan)


In [156]:
OSCSAUDEPARENT['OSCParent2016'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2017, OSCSAUDEPARENT['OSCparent'], np.nan)


In [157]:
OSCSAUDEPARENT['OSCParent2017'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2018, OSCSAUDEPARENT['OSCparent'], np.nan)


In [158]:
OSCSAUDEPARENT['OSCParent2018'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2019, OSCSAUDEPARENT['OSCparent'], np.nan)


In [159]:
OSCSAUDEPARENT['OSCParent2019'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2020, OSCSAUDEPARENT['OSCparent'], np.nan)


In [160]:
OSCSAUDEPARENT['OSCParent2020'] = np.where(OSCSAUDEPARENT['ano_fund'] < 2021, OSCSAUDEPARENT['OSCparent'], np.nan)


In [161]:
def count_before_2015(series):
    return sum(series < 2015)

In [162]:
def count_before_2016(series):
    return sum(series < 2016)

In [163]:
def count_before_2017(series):
    return sum(series < 2017)

In [164]:
def count_before_2018(series):
    return sum(series < 2018)

In [165]:
def count_before_2019(series):
    return sum(series < 2019)

In [166]:
def count_before_2020(series):
    return sum(series < 2020)

In [167]:
def count_before_2021(series):
    return sum(series < 2021)

In [169]:
OSCSAUDEPARENTgrouped = OSCSAUDEPARENT.groupby(['IBGE7']).agg({
    'OSCparent': 'sum',
    'OSCParent2015': 'sum',
    'OSCParent2016': 'sum',
    'OSCParent2017': 'sum',
    'OSCParent2018': 'sum',
    'OSCParent2019': 'sum',
    'OSCParent2020': 'sum',
    'ano_fund': [       count_before_2015,
                        count_before_2016,
                        count_before_2017,
                        count_before_2018,
                        count_before_2019,
                        count_before_2020,
                        count_before_2021]
}).reset_index()

In [170]:
OSCSAUDEPARENTgrouped.columns = ['_'.join(col).strip() for col in OSCSAUDEPARENTgrouped.columns.values]

In [172]:
rename_dict = {
    'OSCparent_sum': 'OSCparent',
    'OSCParentpreref_sum': 'OSCparentpreref',
    'OSCParent2015_sum': 'OSCParent2015',
    'OSCParent2016_sum': 'OSCParent2016',
    'OSCParent2017_sum': 'OSCParent2017',
    'OSCParent2018_sum': 'OSCParent2018',
    'OSCParent2019_sum': 'OSCParent2019',
    'OSCParent2020_sum': 'OSCParent2020',    
    'ano_fund_count_before_2015': 'OSChealthtot2014',
    'ano_fund_count_before_2016': 'OSChealthtot2015',
    'ano_fund_count_before_2017': 'OSChealthtot2016',
    'ano_fund_count_before_2018': 'OSChealthtot2017',
    'ano_fund_count_before_2019': 'OSChealthtot2018',
    'ano_fund_count_before_2020': 'OSChealthtot2019',
    'ano_fund_count_before_2021': 'OSChealthtot2020',
   
}

In [173]:
OSCSAUDEPARENTgrouped.rename(columns=rename_dict, inplace=True)

In [174]:
OSCSAUDEPARENTgrouped

,IBGE7_,OSCparent,OSCParent2015,OSCParent2016,OSCParent2017,OSCParent2018,OSCParent2019,OSCParent2020,OSChealthtot2014,OSChealthtot2015,OSChealthtot2016,OSChealthtot2017,OSChealthtot2018,OSChealthtot2019,OSChealthtot2020
0,1100023,4,3.0,3.0,3.0,4.0,4.0,4.0,4,4,4,4,5,5,5
1,1100049,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1,1
2,1100064,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1,1
3,1100106,1,1.0,1.0,1.0,1.0,1.0,1.0,2,2,2,2,2,2,2
4,1100114,1,1.0,1.0,1.0,1.0,1.0,1.0,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,5221304,1,1.0,1.0,1.0,1.0,1.0,1.0,0,1,1,1,1,1,1
2133,5221403,3,3.0,3.0,3.0,3.0,3.0,3.0,3,3,3,3,4,4,4
2134,5221551,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1,1
2135,5221858,1,1.0,1.0,1.0,1.0,1.0,1.0,2,2,2,2,2,2,2


In [175]:
OSCSAUDEPARENTgrouped.rename(columns={'IBGE7_': 'IBGE7'}, inplace=True)
OSCSAUDEPARENTgrouped['IBGE7'] = OSCSAUDEPARENTgrouped['IBGE7'].astype(int)

In [176]:
OSCSAUDEPARENTgrouped.fillna(0, inplace=True)

In [177]:
OSCSAUDEPARENTgrouped.reset_index(drop=True, inplace=True)

# Mediator variable -  Non-hospital assistance

In [178]:
OSCSAUDEAS = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\Mapa_OSC\Mapa OSC area_subarea.csv', engine = 'python',  delimiter=";", index_col=False, dtype={'IBGE7': np.int64})
OSCSAUDEAS.dropna(subset=['edmu_cd_municipio'], inplace=True)
OSCSAUDEAS.rename(columns={'edmu_cd_municipio': 'IBGE7'}, inplace=True)
OSCSAUDEAS['IBGE7'] = OSCSAUDEAS['IBGE7'].astype(int)
OSCSAUDEAS = OSCSAUDEAS[OSCSAUDEAS['saude'] == 1]

In [179]:
OSCSAUDEAS['Area'] = OSCSAUDEAS['tx_nome_classe_atividade_economica'].apply(lambda x: 1 if x != 'Atividades de atendimento hospitalar' else 0)


In [180]:
OSCSAUDEAS['Year'] = OSCSAUDEAS['dt_fundacao_osc'].str[-4:]

In [181]:
OSCSAUDEASgrouped = OSCSAUDEAS.groupby(['IBGE7','Year']).agg({
    'Area': 'sum',
}).reset_index()

In [182]:
OSCSAUDEASgrouped

,IBGE7,Year,Area
0,1100023,1998,1
1,1100023,2010,1
2,1100023,2012,2
3,1100023,2018,1
4,1100049,2003,0
...,...,...,...
6399,5300108,2016,2
6400,5300108,2017,4
6401,5300108,2018,3
6402,5300108,2019,7


In [183]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2015'] = np.where((OSCSAUDEASgrouped['Year']   < 2015), OSCSAUDEASgrouped['Area'], np.nan)

In [184]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2016'] = np.where((OSCSAUDEASgrouped['Year']   < 2016), OSCSAUDEASgrouped['Area'], np.nan)

In [185]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2017'] = np.where((OSCSAUDEASgrouped['Year']   < 2017), OSCSAUDEASgrouped['Area'], np.nan)

In [186]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2018'] = np.where((OSCSAUDEASgrouped['Year']   < 2018), OSCSAUDEASgrouped['Area'], np.nan)

In [187]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2019'] = np.where((OSCSAUDEASgrouped['Year']   < 2019), OSCSAUDEASgrouped['Area'], np.nan)

In [188]:
OSCSAUDEASgrouped['Year'] = OSCSAUDEASgrouped['Year'].astype(int)
OSCSAUDEASgrouped['OSCSAUDEAS2020'] = np.where((OSCSAUDEASgrouped['Year']   < 2020), OSCSAUDEASgrouped['Area'], np.nan)

In [189]:
OSCSAUDEASgrouped

,IBGE7,Year,Area,OSCSAUDEAS2015,OSCSAUDEAS2016,OSCSAUDEAS2017,OSCSAUDEAS2018,OSCSAUDEAS2019,OSCSAUDEAS2020
0,1100023,1998,1,1.0,1.0,1.0,1.0,1.0,1.0
1,1100023,2010,1,1.0,1.0,1.0,1.0,1.0,1.0
2,1100023,2012,2,2.0,2.0,2.0,2.0,2.0,2.0
3,1100023,2018,1,NaN,NaN,NaN,NaN,1.0,1.0
4,1100049,2003,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6399,5300108,2016,2,NaN,NaN,2.0,2.0,2.0,2.0
6400,5300108,2017,4,NaN,NaN,NaN,4.0,4.0,4.0
6401,5300108,2018,3,NaN,NaN,NaN,NaN,3.0,3.0
6402,5300108,2019,7,NaN,NaN,NaN,NaN,NaN,7.0


In [190]:
OSCSAUDEASgrouped2= OSCSAUDEASgrouped.groupby(['IBGE7']).agg({
    'OSCSAUDEAS2015': 'sum',
    'OSCSAUDEAS2016': 'sum',
    'OSCSAUDEAS2017': 'sum',
    'OSCSAUDEAS2018': 'sum',
    'OSCSAUDEAS2019': 'sum',
    'OSCSAUDEAS2020': 'sum',    
}).reset_index()

In [191]:
OSCSAUDEASgrouped2.fillna(0, inplace=True)

In [192]:
OSCSAUDEASgrouped2

,IBGE7,OSCSAUDEAS2015,OSCSAUDEAS2016,OSCSAUDEAS2017,OSCSAUDEAS2018,OSCSAUDEAS2019,OSCSAUDEAS2020
0,1100023,4.0,4.0,4.0,4.0,5.0,5.0
1,1100049,0.0,0.0,0.0,0.0,0.0,0.0
2,1100064,1.0,1.0,1.0,1.0,1.0,1.0
3,1100106,1.0,1.0,1.0,1.0,1.0,1.0
4,1100114,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
2132,5221304,0.0,1.0,1.0,1.0,1.0,1.0
2133,5221403,2.0,2.0,2.0,2.0,3.0,3.0
2134,5221551,0.0,0.0,0.0,0.0,0.0,0.0
2135,5221858,2.0,2.0,2.0,2.0,2.0,2.0


# Control Variable - Total nonprofit density

In [193]:
OSCTOTAL = pd.read_csv(r'G:\Meu Drive\Doutorado\Disciplinas\Dados\Mapa_OSC\Mapa OSC area_subarea.csv', engine = 'python',  delimiter=";", index_col=False, dtype={'IBGE7': np.int64})
OSCTOTAL.dropna(subset=['edmu_cd_municipio'], inplace=True)
OSCTOTAL.rename(columns={'edmu_cd_municipio': 'IBGE7'}, inplace=True)
OSCTOTAL['IBGE7'] = OSCTOTAL['IBGE7'].astype(int)

In [194]:
OSCTOTAL['ano_fund'] = OSCTOTAL['dt_fundacao_osc'].astype(str).str[-4:]

In [195]:
OSCTOTAL['ano_fund'] = pd.to_numeric(OSCTOTAL['ano_fund'])

In [196]:
OSCTOTALgrouped = OSCTOTAL.groupby(['IBGE7']).agg({
      'ano_fund': [     count_before_2015,
                        count_before_2016,
                        count_before_2017,
                        count_before_2018,
                        count_before_2019,
                        count_before_2020,
                        count_before_2021]
}).reset_index()

In [197]:
OSCTOTALgrouped.columns = ['_'.join(col).strip() for col in OSCTOTALgrouped.columns.values]

In [199]:
rename_dict2 = {
    'ano_fund_count_before_2015': 'OSC2014',
    'ano_fund_count_before_2016': 'OSC2015',
    'ano_fund_count_before_2017': 'OSC2016',
    'ano_fund_count_before_2018': 'OSC2017',
    'ano_fund_count_before_2019': 'OSC2018',
    'ano_fund_count_before_2020': 'OSC2019',
    'ano_fund_count_before_2021': 'OSC2020',
    'IBGE7_': 'IBGE7'
   
}

In [200]:
OSCTOTALgrouped.rename(columns=rename_dict2, inplace=True)

In [201]:
OSCTOTALgrouped.fillna(0, inplace=True)

In [202]:
OSCTOTALgrouped.reset_index(drop=True, inplace=True)

In [203]:
OSCTOTALgrouped

,IBGE7,OSC2014,OSC2015,OSC2016,OSC2017,OSC2018,OSC2019,OSC2020
0,1100015,117,118,122,125,129,131,134
1,1100023,401,429,439,458,479,491,498
2,1100031,36,39,39,40,42,42,43
3,1100049,297,307,317,327,340,354,358
4,1100056,74,77,85,89,92,95,95
...,...,...,...,...,...,...,...,...
5565,5222005,50,54,54,54,55,56,57
5566,5222054,15,16,16,16,17,17,18
5567,5222203,9,10,12,15,16,16,16
5568,5222302,17,20,21,22,22,22,23


# Joining variables

In [204]:
Dataset = pd.merge(dadosibge, Pop, how = 'left', on = 'IBGE7')

In [205]:
Dataset1 = pd.merge(Dataset, CNESOSC, how = 'left', on = 'IBGE')

In [206]:
Dataset2 = pd.merge(Dataset1, pivot_FNStransfersOSC, how = 'left', on = 'IBGE')

In [207]:
Dataset2['IBGE7'] = Dataset2['IBGE7'].astype(int)
Dataset3 = pd.merge(Dataset2, OSCSAUDEPARENTgrouped, how = 'left', on = 'IBGE7')

In [208]:
Dataset3['IBGE7'] = Dataset3['IBGE7'].astype(int)
Dataset4 = pd.merge(Dataset3, Eco, how = 'left', on = 'IBGE7')

In [209]:
Dataset5 = pd.merge(Dataset4, Cadunico, how = 'left', on = 'IBGE')

In [210]:
Dataset6 = pd.merge(Dataset5, Sigafiltered2group, how = 'left', on = 'IBGE7')

In [211]:
Dataset7= pd.merge(Dataset6, OSCSAUDEASgrouped2, how = 'left', on = 'IBGE7')

In [212]:
Dataset8 = pd.merge(Dataset7, OSCTOTALgrouped, how = 'left', on = 'IBGE7')

In [213]:
Dataset8.fillna(0, inplace=True)

In [214]:
Dataset8

,IBGE,IBGE7,AmazoniaLegal2015,Semiarido2015,Centrolocal,CentroSubregional,CapitalRegional,CentroZona,Nucleo,MUNICIPIO,...,OSCSAUDEAS2018,OSCSAUDEAS2019,OSCSAUDEAS2020,OSC2014,OSC2015,OSC2016,OSC2017,OSC2018,OSC2019,OSC2020
0,110001,1100015,1,-1,1,-1,-1,-1,-1,ALTA FLORESTA DO OESTE,...,0.0,0.0,0.0,117,118,122,125,129,131,134
1,110002,1100023,1,-1,-1,1,-1,-1,-1,ARIQUEMES,...,4.0,5.0,5.0,401,429,439,458,479,491,498
2,110003,1100031,1,-1,1,-1,-1,-1,-1,CABIXI,...,0.0,0.0,0.0,36,39,39,40,42,42,43
3,110004,1100049,1,-1,-1,-1,1,-1,-1,CACOAL,...,0.0,0.0,0.0,297,307,317,327,340,354,358
4,110005,1100056,1,-1,-1,-1,-1,1,-1,CEREJEIRAS,...,0.0,0.0,0.0,74,77,85,89,92,95,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,522200,5222005,-1,-1,1,-1,-1,-1,-1,VIANOPOLIS,...,0.0,0.0,0.0,50,54,54,54,55,56,57
5566,522205,5222054,-1,-1,1,-1,-1,-1,-1,VICENTINOPOLIS,...,0.0,0.0,0.0,15,16,16,16,17,17,18
5567,522220,5222203,-1,-1,1,-1,-1,-1,-1,VILA BOA,...,0.0,0.0,0.0,9,10,12,15,16,16,16
5568,522230,5222302,-1,-1,1,-1,-1,-1,-1,VILA PROPICIO,...,0.0,0.0,0.0,17,20,21,22,22,22,23


In [215]:
csv_file_path = r"C:\Users\lunav\Downloads\Dataset8.csv"

Dataset8.to_csv(csv_file_path, index=False)